In [1]:
#import tensorflow as tf
import os
import numba
import random
import numpy as np
#import scipy.special
import scipy.ndimage.interpolation as sni
#import numpy.linalg as lg
import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2d
%matplotlib inline

def ReLU(x):
    return np.maximum(0, x)

def D_ReLU(x):
    return np.where(x < 0, 0, 1)

def L_ReLU(x):
    return np.where(x < 0, 0.001 * x, x)

def D_L_ReLU(x):
    return np.where(x < 0, 0.001, 1)

def I_L_ReLU(x):
    return np.where(x < 0, 1000 * x, x)

def softmax(x):
    for i in range(x.shape[1]):
        x[:, i] -= np.max(x[:, i])
        x[:, i] = np.exp(x[:, i])
        x[:, i] = x[:, i] / np.sum(x[:, i])
    return x

# 交叉熵损失函数
# cross-entropy cost function
def Loss(predicted, target):
    return softmax(predicted) - target

def mkdir(path):
    folder = os.path.exists(path)
    #判断是否存在文件夹如果不存在则创建为文件夹 If the folder does not exist, create a new folder
    if not folder:                   
        os.makedirs(path)            
        
@numba.njit
def RandomNoise(x):
    x = x.reshape(-1) # flat view
    for ii in range(len(x)):
        x[ii] += random.uniform(-0.1, 0.1)

In [2]:
# 自定义卷积函数
# Convolution function

def convolution2D(input_array, kernel, stride):
    input_row, input_column = input_array.shape #行 列
    kernel_row, kernel_column = kernel.shape
    new_row = (input_row - kernel_row) // stride + 1

    new_column = (input_column - kernel_column) // stride + 1
    new_array = np.zeros((new_row, new_column))

    for j in range(new_row):
        for k in range(new_column):
            new_array[j, k] = np.sum(input_array[j * stride : j * stride + kernel_row, \
                                                        k * stride : k * stride + kernel_column] * kernel)
    
    return new_array


# 零填充
# zero padding
def padding(input_array, n):
    if n == 0:
        return input_array
    else:
        input_array_size = input_array.shape
        output_array = np.zeros((input_array_size[0], input_array_size[1] + 2 * n, input_array_size[2] + 2 * n, input_array_size[3]))
        output_array[:, n : n + input_array_size[1], n : n + input_array_size[2], :] = input_array
        return output_array

In [3]:
# 卷积层
# Convolution layer
class ConvLayer:
    def __init__(self, input_shape, kernel_shape, stride, zero_padding, learning_rate, \
                 residual_error_flag, first_layer_flag, lock_flag, name):
                
        # 样本数 行 列 通道数
        # sample size， row, column. channel
        self.input_shape = input_shape
        self.input_number = input_shape[0] #minibatch
        
        self.input_row = input_shape[1] + 2 * zero_padding
        self.input_column = input_shape[2] + 2 * zero_padding
        self.input_channel = input_shape[3] 
                   
        self.kernel_shape = kernel_shape 
        #行 列 输入通道数 卷积核通道数
        #row, column, input channel, kernel
        
        self.kernel_size = kernel_shape[0]
        self.kernel_channel = kernel_shape[3]
        self.stride = stride 
        self.zero_padding = zero_padding
        self.learning_rate = learning_rate * (1 / self.input_number)
        self.residual_error_flag = residual_error_flag
        self.first_layer_flag = first_layer_flag
        self.lock_flag = lock_flag
        self.name = name
        
        self.conv_out_row = (self.input_row - self.kernel_size) // self.stride + 1     
        self.conv_out_column = (self.input_column - self.kernel_size) // self.stride + 1
        #self.conv_out_array = np.zeros((self.input_number, self.conv_out_row, self.conv_out_column, self.kernel_channel))
        
        # kaiming高斯初始化
        # kaiming Gaussian initialization
        #self.kernel = np.random.normal(0.0, pow(2 / (self.kernel_size * self.kernel_size * self.kernel_channel), 0.5), \
                                           #(self.kernel_size, self.kernel_size, self.kernel_channel))
        
        # xavier初始化
        # xavier initialization
        input_number = self.input_row * self.input_column * self.input_channel
        conv_out_number = self.conv_out_row * self.conv_out_column * self.kernel_channel
        
        temp_number = pow(6 / (input_number + conv_out_number), 0.5)
        
        self.kernel = np.random.uniform(-temp_number, temp_number, kernel_shape)
        #self.kernel_error = np.zeros(kernel_shape)
        
        self.activation_function = lambda x : L_ReLU(x)
        self.D_activation_function = lambda x : D_L_ReLU(x)
        
    # 正向查询
    # forward propagation
    def fp(self, input_array):
        
        input_array = padding(input_array, self.zero_padding)
        
        self.conv_out_array = np.zeros((input_array.shape[0], self.conv_out_row, self.conv_out_column, self.kernel_channel))
        # 卷积操作 Convolution
        for n in range(input_array.shape[0]):
            for i in range(self.kernel_channel):
                #temp_kernel = np.dstack([self.kernel[:, :, i]] * self.input_channel)
                for j in range(self.conv_out_row):
                    for k in range(self.conv_out_column):
                        self.conv_out_array[n, j, k, i] = \
                        np.sum(input_array[n, j * self.stride : j * self.stride + self.kernel_size, \
                                           k * self.stride : k * self.stride + self.kernel_size, :] * self.kernel[:, :, :, i])
        if self.residual_error_flag:
            return self.activation_function(self.conv_out_array) + self.conv_out_array
                    
        return self.activation_function(self.conv_out_array)
    
    
    
    # 反向查询
    # back propagation
    def bp(self, input_array, error):
        
        input_array = padding(input_array, self.zero_padding)
        
        if self.residual_error_flag:
            conv_out_error = error * (self.D_activation_function(self.conv_out_array) + 1)
        else:
            conv_out_error = error * self.D_activation_function(self.conv_out_array)

        new_array_row = (self.conv_out_row - 1) * self.stride + 1
        new_array_column = (self.conv_out_column - 1) * self.stride + 1
            
        if self.stride != 1:     
            
            
            new_array = np.zeros((self.input_number, new_array_row, new_array_column, self.kernel_channel))
            for n in range(self.input_number):
                for i in range(self.conv_out_row):
                    for j in range(self.conv_out_column):
                        new_array[n, i * self.stride , j * self.stride , :] = conv_out_error[n, i, j, :]
        else:
            new_array = conv_out_error
        
        
        
        if self.first_layer_flag :
            conv_error = None
        else:
            new_array_padding = padding(new_array, self.kernel_size - 1)
            conv_error = np.zeros(input_array.shape)
            for n in range(self.input_number):
                for i in range(self.kernel_channel):
                    for j in range(self.input_channel):
                        conv_error[n, :, :, j] += conv2d(new_array_padding[n, :, :, i], self.kernel[:, :, j, i], mode='valid')
            
            conv_error = conv_error[:, self.zero_padding : self.input_row - self.zero_padding, \
                          self.zero_padding : self.input_column - self.zero_padding, :]
            
            
        if not self.lock_flag:            
            kernel_error = np.zeros(self.kernel_shape)

            for n in range(self.input_number):
                for i in range(self.kernel_channel):
                    for j in range(self.input_channel):
                        kernel_error[:, :, j, i] += convolution2D(input_array[n, :, :, j], new_array[n, :, :, i], 1)

            self.kernel -= self.learning_rate * kernel_error     
                     
        return conv_error
    
    
    def backQuery(self, error):
        # 用于可视化反卷积 Used for visualization
        # Matthew D. Zeiler , Rob Fergus. Visualizing and Understanding Convolutional Networks.European Conference on Computer Vision, 2014 – Springer.
        
        new_array_padding = padding(error, self.kernel_size - 1)

        conv_error = np.zeros((1, self.input_row, self.input_column, self.input_channel))
        for n in range(1):
            for i in range(self.kernel_channel):
                for j in range(self.input_channel):
                    conv_error[n, :, :, j] += conv2d(new_array_padding[n, :, :, i], self.kernel[:, :, j, i], mode='valid')

        conv_error = conv_error[:, self.zero_padding : self.input_row - self.zero_padding, \
                        self.zero_padding : self.input_column - self.zero_padding, :]
        
            
            
            
        return self.activation_function(conv_error)

    
    
    def saveKernel(self, path):
        path += self.name
        mkdir(path)
        for i in range(self.kernel_channel):
            for j in range(self.input_channel):
                filename = path + '\\kernel_' + str(i) + str(j) + '.txt'
                np.savetxt(filename, self.kernel[:, :, j, i], fmt='%2.54f', delimiter=",")
        
    def loadKernel(self, path):
        path += self.name
        for i in range(self.kernel_channel):
            for j in range(self.input_channel):
                filename = path + '\\kernel_' + str(i) + str(j) + '.txt'
                self.kernel[:, :, j, i] = np.loadtxt(filename, dtype=np.float64, delimiter=',')

In [4]:
# 池化层
class PoolingLayer:

    def __init__(self, input_shape, kernel_size, stride, mode):
        
        self.input_number, self.input_row, self.input_column, self.input_channel = input_shape #行 列 通道数
        self.kernel_size = kernel_size
        self.stride = stride
        self.mode = mode
               
        self.pooling_out_row = (self.input_row - self.kernel_size) // self.stride + 1
        self.pooling_out_column = (self.input_column - self.kernel_size) // self.stride + 1
        #self.pooling_out_array = np.zeros((self.input_number, self.pooling_out_row, self.pooling_out_column, self.input_channel))
        
        self.pooling_error = np.zeros(input_shape)
        
        self.avgpooling_parameter = 1 / (kernel_size * kernel_size)
        
        # 位置矩阵， 保存最大池化的位置信息 Save location information of max pooling
        self.location_array = np.zeros((self.input_number, self.pooling_out_row, self.pooling_out_column, \
                                        self.input_channel * 2)).astype(np.int32)
        
    # 正向查询
    def fp(self, input_array):
        
        self.pooling_out_array = np.zeros((input_array.shape[0], self.pooling_out_row, self.pooling_out_column, self.input_channel))
        # 池化操作
        if self.mode == 'max':
            for n in range(input_array.shape[0]):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                            temp_location = \
                            np.argmax(input_array[n, j * self.stride : j * self.stride + self.kernel_size, \
                                                  k * self.stride : k * self.stride + self.kernel_size, i])
                            self.location_array[n, j, k, 2 * i] = j * self.stride + temp_location // self.kernel_size
                            self.location_array[n, j, k, 2 * i + 1] = k * self.stride + temp_location % self.kernel_size
                            self.pooling_out_array[n, j, k, i] = input_array[n, self.location_array[n, j, k, 2 * i], \
                                                                             self.location_array[n, j, k, 2 * i + 1], i]
        elif self.mode == 'avg':
            for n in range(input_array.shape[0]):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                            self.pooling_out_array[n, j, k, i] = self.avgpooling_parameter * \
                                        np.sum(input_array[n, j * self.stride : j * self.stride + self.kernel_size, \
                                                           k * self.stride : k * self.stride + self.kernel_size, i])

        else:
            raise Exception('mode Error')  
            
            
            
        return self.pooling_out_array
    
    def bp(self, error):
        
        if self.mode == 'max':
            for n in range(self.input_number):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                             self.pooling_error[n, self.location_array[n, j, k, 2 * i], self.location_array[n, j, k, 2 * i + 1], i] = \
                                error[n, j, k, i]

        elif self.mode == 'avg':
            for n in range(self.input_number):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                             self.pooling_error[n, j * self.stride : j * self.stride + self.kernel_size, \
                                                k * self.stride : k * self.stride + self.kernel_size, i] += \
                                                self.avgpooling_parameter * error[n, j, k, i]
        else:
            raise Exception('mode Error') 
            
        return self.pooling_error
    def backQuery(self, error):
        if self.mode == 'max':
            for n in range(1):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                             self.pooling_error[n, self.location_array[n, j, k, 2 * i], self.location_array[n, j, k, 2 * i + 1], i] = \
                                error[n, j, k, i]
        return self.pooling_error

In [5]:
# 全连接层
class FullyConnectedLayer:
    def __init__(self, input_nodes, output_nodes, learning_rate, mini_batch, residual_error_flag, lock_flag, name):
        
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate * (1 / mini_batch)
        self.mini_batch = mini_batch
        self.residual_error_flag = residual_error_flag
        self.lock_flag = lock_flag
        self.name = name
        
        #self.bias = np.ones((1, 1))
        
        # 高斯初始化权重
        #self.weight = np.random.normal(0.0, pow(2 / input_nodes, 0.5), (output_nodes, input_nodes))
        
        # xavier初始化
        temp_number = pow(6 / (self.input_nodes + self.output_nodes), 0.5)
        self.weight = np.random.uniform(-temp_number, temp_number, (self.output_nodes, self.input_nodes))
        
        # 激活函数
        self.activation_function = lambda x : L_ReLU(x)
        self.D_activation_function = lambda x : D_L_ReLU(x)
        
        
        
        
    def fp(self, input_array):
         
        self.output_array = np.dot(self.weight, input_array)
        
        if self.residual_error_flag:
            return self.activation_function(self.output_array) + self.output_array
        
        return self.activation_function(self.output_array)
    
    def bp(self, input_array, error):
        
        if self.residual_error_flag:
            output_error = error * (self.D_activation_function(self.output_array) + 1)
        else:
            output_error = error * self.D_activation_function(self.output_array)
        input_error = np.dot(self.weight.T, output_error)
        
        if not self.lock_flag:
            self.weight -= self.learning_rate * np.dot(output_error, np.transpose(input_array))
        
        return input_error
    
    def saveWeight(self, path):
        path += self.name
        mkdir(path)
        filename = path + '\\weight.txt'
        np.savetxt(filename, self.weight, fmt='%2.54f', delimiter=",")
        
    def loadWeight(self, path):
        path += self.name
        filename = path + '\\weight.txt'
        self.weight = np.loadtxt(filename, dtype=np.float64, delimiter=',')     
    
    
    
        
        
        

In [6]:
# 全连接层
# Same thing as FullyConnectedLayer
class LastConnectedLayer:
    def __init__(self, input_nodes, output_nodes, learning_rate, mini_batch, residual_error_flag, lock_flag, name):
        
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate * (1 / mini_batch)
        self.mini_batch = mini_batch
        self.residual_error_flag = residual_error_flag
        self.lock_flag = lock_flag
        self.name = name
        
        #self.bias = np.ones((1, 1))
        
        # 高斯初始化权重
        #self.weight = np.random.normal(0.0, pow(2 / input_nodes, 0.5), (output_nodes, input_nodes))
        
        # xavier初始化
        temp_number = pow(6 / (self.input_nodes + self.output_nodes), 0.5)
        self.weight = np.random.uniform(-temp_number, temp_number, (self.output_nodes, self.input_nodes))
        
        # 激活函数
        self.activation_function = lambda x : L_ReLU(x)
        self.D_activation_function = lambda x : D_L_ReLU(x)
        
        
        
        
    def fp(self, input_array):
         
        self.output_array = np.dot(self.weight, input_array)
              
        return self.output_array
    
    def bp(self, input_array, error):
        
        input_error = np.dot(self.weight.T, error)
        
        if not self.lock_flag:
            self.weight -= self.learning_rate * np.dot(error, np.transpose(input_array))
        
        return input_error
    
    def saveWeight(self, path):
        path += self.name
        mkdir(path)
        filename = path + '\\weight.txt'
        np.savetxt(filename, self.weight, fmt='%2.54f', delimiter=",")
        
    def loadWeight(self, path):
        path += self.name
        filename = path + '\\weight.txt'
        self.weight = np.loadtxt(filename, dtype=np.float64, delimiter=',')     
    
    
    
        
        
        

In [7]:
# 批标准化 Batch Normalization
# 论文作者的更复杂，此程序简化了反向传播时的一些计算量
# Reduced some computation of back propagation
# Sergey Ioffe, Christian Szegedy. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift.International Conference on Machine Learning

class BatchNormalizationLayer2D:
    def __init__(self, input_shape, learning_rate, lock_flag, name):
        self.input_row, self.input_column = input_shape
        self.learning_rate = learning_rate * (1 / self.input_column)
        self.lock_flag = lock_flag
        self.name = name
        self.mean = np.zeros((self.input_row, 1))
        self.variance = np.ones((self.input_row, 1))        
        self.beta = np.zeros((self.input_row, 1))
        self.gamma = np.ones((self.input_row, 1))
        self.momentum = 0.999

    def fp(self, input_array):
        if not self.lock_flag:
            if input_array.shape[1] != 1:
                self.mean = self.momentum * self.mean + (1 - self.momentum) * input_array.mean(axis = 1).reshape(self.input_row, 1)
                self.variance = self.momentum  * self.variance + \
                                          (1.0 - self.momentum) * input_array.var(axis = 1).reshape(self.input_row, 1)
        #Xi
        self.new_array = (input_array - self.mean) / np.sqrt(self.variance + 0.000001)
        self.output_array = self.new_array * self.gamma + self.beta
        return self.output_array
    
    def bp(self, error):
        
        new_array_error = error * self.gamma   
        if not self.lock_flag:
            self.gamma -= self.learning_rate * np.sum((error * self.new_array), axis=1).reshape(self.input_row, 1)        
            self.beta -= self.learning_rate * np.sum(error, axis=1).reshape(self.input_row, 1)
        input_error = new_array_error / np.sqrt(self.variance + 0.000001)
        
        return input_error
        
        


        
    def saveBNParameter(self, path):
        path += self.name
        mkdir(path)
        filename = path + '\\mean.txt'
        np.savetxt(filename, self.mean.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
        filename = path + '\\variance.txt'
        np.savetxt(filename, self.variance.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
        filename = path + '\\gamma.txt'
        np.savetxt(filename, self.gamma.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
        filename = path + '\\beta.txt'
        np.savetxt(filename, self.beta.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
    
    def loadBNParameter(self, path):
        path += self.name
        filename = path + '\\mean.txt'
        self.mean = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        filename = path + '\\variance.txt'
        self.variance = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        filename = path + '\\gamma.txt'
        self.gamma = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        filename = path + '\\beta.txt'
        self.beta = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        



In [8]:
# SoftMax层
class SoftMax:
    def fp(self, input_array):
        return softmax(input_array)
    def train(self, input_array, target_array):
        return Loss(input_array, target_array)

In [9]:
# 打开mnist数据集，加载到一个60000 * 28 * 28 * 1的矩阵中
# load the mnist data set into a matrix
training_data_file = open("D:/python/neural_network/mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

test_data_file = open("D:/python/neural_network/mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

training_data_array = np.zeros((60000, 28, 28, 1))
training_label_list = []
for i, record in enumerate(training_data_list):
    all_values = record.split(',')
    training_data_array[i, :, :, :] = np.asfarray(all_values[1:]).reshape(28, 28, 1)
    training_label_list.extend(all_values[0])
training_data_array /= 255.0
training_data_array *= 0.99
training_data_array += 0.01


del training_data_list

test_data_array = np.zeros((10000, 28, 28, 1))
test_label_list = []
for i, record in enumerate(test_data_list):
    all_values = record.split(',')
    test_data_array[i, :, :, :] = np.asfarray(all_values[1:]).reshape(28, 28, 1)
    test_label_list.extend(all_values[0])
test_data_array /= 255.0
test_data_array *= 0.99
test_data_array += 0.01

del test_data_list
    





In [10]:
minibatch = 32
class CNN:
    def __init__(self):
        self.layer1 = ConvLayer((minibatch, 28, 28, 1), (5, 5, 1, 6), 1, 0, 0.0003, True, True, False, 'layer1')
        self.layer2 = PoolingLayer((minibatch, 24, 24, 6), 2, 2, 'max')
        self.layer3 = ConvLayer((minibatch, 12, 12, 6), (3, 3, 6, 20), 1, 0, 0.0003, True, False, False, 'layer3')
        self.layer4 = PoolingLayer((minibatch, 10, 10, 20), 2, 2, 'max')
        self.layer5 = FullyConnectedLayer(500, 120, 0.1, minibatch, False, False, 'layer5')
        self.layerx = BatchNormalizationLayer2D((120, 32), 0.1, False, 'layerx')
        self.layer6 = FullyConnectedLayer(120, 80, 0.1, minibatch, False, False, 'layer6')
        self.layer7 = LastConnectedLayer(80, 10, 0.1, minibatch, False, False, 'layer7')
        self.layer8 = SoftMax()
        
    def query(self, input_array):
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        out3 = self.layer3.fp(out2)
        out4 = self.layer4.fp(out3).reshape(500, 1)
        out5 = self.layer5.fp(out4)
        outx = self.layerx.fp(out5)
        out6 = self.layer6.fp(outx)
        out7 = self.layer7.fp(out6)
        out8 = self.layer8.fp(out7)
        return out8
    def train(self, input_array, target_array):
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        out3 = self.layer3.fp(out2)
        out4 = self.layer4.fp(out3).reshape(minibatch, 500).T
        out5 = self.layer5.fp(out4)
        outx = self.layerx.fp(out5)
        out6 = self.layer6.fp(outx)
        out7 = self.layer7.fp(out6)
        error1 = self.layer8.train(out7, target_array)
        error2 = self.layer7.bp(out6, error1)
        error3 = self.layer6.bp(outx, error2)
        errorx = self.layerx.bp(error3)
        error4 = self.layer5.bp(out4, errorx).T.reshape(minibatch, 5, 5, 20)
        error5 = self.layer4.bp(error4)
        error6 = self.layer3.bp(out2, error5)
        error7 = self.layer2.bp(error6)
        error8 = self.layer1.bp(input_array, error7)
    
    # 用于可视化，可忽略
    # Visualization, can be ignored
    def layer1bp(self, input_array, n):
        out1 = self.layer1.fp(input_array)
        
        for i in range(out1.shape[3]):
            if n == i:
                continue
            else:
                out1[:, :, :, i] = 0
        xxxx = self.layer1.backQuery(out1)
        return xxxx
    
    # 用于可视化，可忽略
    # Visualization, can be ignored
    def layer2bp(self, input_array, n):
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        
        for i in range(out2.shape[3]):
            if n != i:
 
                out2[:, :, :, i] = 0
        x1 = self.layer2.backQuery(out2)
        x2 = self.layer1.backQuery(x1)
        return x2
    
    # 用于可视化，可忽略
    # Visualization, can be ignored
    def layer3bp(self, input_array, n):
        #fig = plt.figure()
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        out3 = self.layer3.fp(out2)
        
        for i in range(out3.shape[3]):
            if n == i:
                continue
            else:
                out3[:, :, :, i] = 0

        x0 = self.layer3.backQuery(out3)
        x1 = self.layer2.backQuery(x0)
        x2 = self.layer1.backQuery(x1)
        
        return x2
    
    def saveData(self, path):
        self.layer1.saveKernel(path)
        self.layer3.saveKernel(path)
        self.layer5.saveWeight(path)
        self.layer6.saveWeight(path)
        self.layer7.saveWeight(path)
        self.layerx.saveBNParameter(path)
        
    def loadData(self, path):
        self.layer1.loadKernel(path)
        self.layer3.loadKernel(path)
        self.layer5.loadWeight(path)
        self.layer6.loadWeight(path)
        self.layer7.loadWeight(path)
        self.layerx.loadBNParameter(path)

In [11]:
cnn = CNN()

In [16]:
a = [-10, 0, 10] #图片旋转角度 Image rotation Angle

i = 0
j = 0
k = 0
q = 0
minibatch = 32

# 训练
input_array = np.zeros((minibatch, 28, 28, 1))
target_list = np.zeros((10, minibatch))
for n in range(60000):
    inputs = training_data_array[n, :, :, :]
    inputs = sni.rotate(inputs.reshape(28, 28), random.sample(a, 1)[0], cval=0.01, reshape=False)
    RandomNoise(inputs)
    input_array[i, :, :, 0] = inputs
    #print(np.max(inputs))
    #print(np.min(inputs))
    targets = np.zeros(10)
    targets[int(training_label_list[n])] = 1
    target_list[:, i] = targets
    i += 1
    if i == minibatch:
        cnn.train(input_array, target_list)
        i = 0
    j += 1    
    if j == 100:
        j = 0
        print(k)
        print(cnn.layer1.kernel[0, 0, 0, 0])
        #print(cnn.layery.variance[0, 0, 0, 0])
        print(cnn.layer3.kernel[0, 0, 0, 0])
        print(cnn.layer5.weight[0, 0])
        print(cnn.layer7.weight[0, 0])
        #print(cnn.layerx.variance[0, 0])
        k += 1

    





0
-0.00131518451376669
0.02014177228014032
0.019431401831009023
-0.03386806087790044
1
-0.0013333364125700355
0.020144521615313906
0.019443106381748846
-0.03373570335042507
2
-0.0013546671921601712
0.02014211940068342
0.019441019192275123
-0.033423962291092485
3
-0.0014162216597067077
0.02014282710101701
0.019438218929064294
-0.03344978072426692
4
-0.001433057340834134
0.020143094046722782
0.019422183371744887
-0.033198587092487665
5
-0.0014242071849730462
0.020143511478066815
0.01940594908359932
-0.03313125216007296
6
-0.0014064834508210966
0.020145028089820192
0.01938355756032454
-0.03314167482679116
7
-0.0014161819676739881
0.020145792256286398
0.019373546279194083
-0.03312848055967908
8
-0.0014106880529245287
0.02014924427162839
0.019391869373234526
-0.0331212068240171
9
-0.001480674324249843
0.02015457165325758
0.019382239912813005
-0.033119083774214936
10
-0.0015732768324897648
0.020157885141285074
0.019373919966087696
-0.033036166577403575
11
-0.001595209937558848
0.020163288439

94
-0.009573872825397808
0.020523607972124523
0.017350233911057
-0.023015210504999845
95
-0.009697203483753516
0.020535466586665697
0.017236126419835827
-0.024065696554751444
96
-0.009870377839860722
0.020541154346382002
0.017224537686494127
-0.02326346535911116
97
-0.009994045500563408
0.020525318866638748
0.017156052593340666
-0.022397692367729566
98
-0.010038853550147676
0.02053870576870182
0.017101253573164926
-0.022150361520040948
99
-0.010112433795631228
0.020575115446118328
0.017044616087616638
-0.021946511349480353
100
-0.01008408333716345
0.02059813899300828
0.01700194641140966
-0.02329671507475769
101
-0.009956609286011522
0.020626058943415262
0.016986431849621796
-0.02194084673317494
102
-0.0097225635366356
0.020673148039482555
0.016943236967825872
-0.020232114669595073
103
-0.009445321993422611
0.020732405479895223
0.01689999156195707
-0.022676284157835767
104
-0.009413939565137305
0.02076129699882186
0.01686971006449909
-0.02535376710571569
105
-0.009509057306379635
0.0207

187
-0.010006083616203709
0.02230378251013543
0.015074559059335808
-0.07056885895841537
188
-0.009830960864769537
0.022257449449466785
0.015105877387982632
-0.06826770082682806
189
-0.0100542167835545
0.02222127016643487
0.015114230293705988
-0.0689338010906797
190
-0.009926927296862611
0.022227349814687408
0.015085199541214995
-0.06753057026834793
191
-0.00998743826839535
0.022261330401725406
0.015067610540890121
-0.06993560096729275
192
-0.009560748056127721
0.022283659783655554
0.015044979911093798
-0.07076726144699245
193
-0.00952369280057915
0.022286635545105286
0.014983692085199273
-0.06726214447028304
194
-0.009314280712793502
0.022279326525158346
0.015000898627970572
-0.06764501344645206
195
-0.009284319202290382
0.02227327523673983
0.014985216793086436
-0.0692468090629712
196
-0.009245822781472355
0.022258179369465926
0.014953666726050644
-0.0667219703930874
197
-0.009017340846061341
0.022264310605915265
0.01497707037597701
-0.06820408050378225
198
-0.008367077947836487
0.0222

280
-0.0032433720012357164
0.02259258249302863
0.014901053651579725
-0.08186249790227282
281
-0.003119373722827899
0.022656832160779752
0.014919774865976393
-0.08401850090665915
282
-0.0027929822273098804
0.022746369280900975
0.01489353606344003
-0.08521883891651688
283
-0.002044954117036451
0.022795022131786996
0.01488787271507704
-0.083439765304307
284
-0.0026881333197238105
0.022780381161813024
0.014930941888068346
-0.0836128623815737
285
-0.0030232717325913475
0.022787564402302746
0.01516958695305124
-0.08369987032329812
286
-0.0027870914823610077
0.022688629062542556
0.015167288847271287
-0.08414332727309659
287
-0.0032836161987865424
0.02273834459428262
0.015100731316349786
-0.08551047063837065
288
-0.003872029636642618
0.022758160915491127
0.01513512776764426
-0.0876413398106303
289
-0.004551302999891925
0.02272321739391492
0.01510608665579594
-0.08502083105815751
290
-0.005009921041653945
0.022688196123651326
0.015086183426244095
-0.0825243946261382
291
-0.0050575927881001004
0

373
-0.00365773449845414
0.023944973090655082
0.01504955893897339
-0.08400778967466718
374
-0.003796848909120824
0.023828527712285403
0.015073200998067401
-0.08587874007591312
375
-0.004243647123713959
0.023756636101714884
0.015204974598179153
-0.08593669682464833
376
-0.00386326175621668
0.023687071527929374
0.015312306036000057
-0.08607794952466666
377
-0.004019935730690386
0.023714002266077006
0.01545258981570533
-0.08478909134333579
378
-0.005254941523206182
0.023639680433375634
0.015509860757532997
-0.0840604953651965
379
-0.006212701936492573
0.023648916663376356
0.015505477426823958
-0.0842219929650609
380
-0.006389061323998109
0.023713166883781275
0.015522775099971579
-0.08433375038939696
381
-0.006232542217970215
0.023723060372916566
0.015552510012706206
-0.08718737779915475
382
-0.006011452113288327
0.023782615613239168
0.015577004836988711
-0.08970149626022035
383
-0.005778329671111185
0.02381038190402339
0.01553734606013649
-0.09033108995561763
384
-0.005984679777919056
0.0

466
-0.007986749084328722
0.02391373692954469
0.016085905559864198
-0.10511749458812758
467
-0.007989665586440457
0.023977697987184764
0.015729289361293253
-0.10316334236553179
468
-0.008530575908280852
0.023995500060673962
0.015627633996602865
-0.10196552949376456
469
-0.00845757346309218
0.023950905780432734
0.015703210061814302
-0.10208915310253226
470
-0.008862976461470181
0.023940202983557684
0.015613094510552442
-0.10230068464171446
471
-0.008634707108514324
0.023860179332701313
0.01576068039479266
-0.10278043253000597
472
-0.008089563190941085
0.02382825815301623
0.015579915037968371
-0.10282801546063129
473
-0.007814587922356772
0.023825372024707622
0.015572765232940761
-0.10419247192354208
474
-0.0075955942617303525
0.02375791474933478
0.015630589099777575
-0.10400640326657687
475
-0.007965190391776716
0.02368832942669577
0.015728882287255237
-0.10345525921773016
476
-0.007543998579051519
0.023788508350933005
0.01569143478823618
-0.10650396732463924
477
-0.00656205303371932
0.

560
-0.004265376255909823
0.025737769560480853
0.0186028978784091
-0.12005993824665354
561
-0.003538009837807035
0.025697256278041654
0.018586122006996306
-0.11782765629447588
562
-0.00350997852160694
0.02577380841109164
0.018586556198462896
-0.11714179959616831
563
-0.0035870523578419374
0.025831630334853254
0.0186320642375956
-0.11650438780108567
564
-0.003074723386969503
0.025789325270904568
0.018473080629064057
-0.11242303911107998
565
-0.0024572300557194906
0.025756762914841594
0.018461911378202703
-0.11144192924276243
566
-0.0024477858607408723
0.025804174978379647
0.018614106873766467
-0.11056522124819065
567
-0.0025022274675239988
0.025895682129514316
0.018696844894302
-0.11054468186851778
568
-0.002675505178610818
0.025917122846926773
0.0186585482421195
-0.11169995367316218
569
-0.0032422389322192212
0.025981599833335253
0.01862023267966467
-0.11175613780022634
570
-0.004130161619614682
0.025950903572964673
0.01877988155807525
-0.11198238391198824
571
-0.004786178823064278
0.0

In [14]:

scorecard = [] 
for i, correct_label in enumerate(test_label_list[0*1000:10*1000]): 
    inputs = test_data_array[i, :, :, :]
    inputs = inputs.reshape(1, 28, 28, 1) 
    outputs = cnn.query(inputs) 
    #print(outputs)
    label = np.argmax(outputs) 
    #print(label, correct_label)

    if(label == int(correct_label)): 
        scorecard.append(1) 
    else: 
        scorecard.append(0)

#print("test_T_F_list", scorecard) 
scorecard_array = np.asarray(scorecard) 
print("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9909


In [ ]:
a = [-10, 0, 10]
path = 'D:\\python\\neural_network\\parameter37\\count_'
counter = 0
while 1 :
    a = [-10, 0, 10]
    i = 0
    j = 0
    k = 0
    q = 0
    minibatch = 32

    # 训练
    input_array = np.zeros((minibatch, 28, 28, 1))
    target_list = np.zeros((10, minibatch))
    for n in range(60000):
        inputs = training_data_array[n, :, :, :]
        inputs = sni.rotate(inputs.reshape(28, 28), random.sample(a, 1)[0], cval=0.01, reshape=False)
        RandomNoise(inputs)
        input_array[i, :, :, 0] = inputs
        #print(np.max(inputs))
        #print(np.min(inputs))
        targets = np.zeros(10)
        targets[int(training_label_list[n])] = 1
        target_list[:, i] = targets
        i += 1
        if i == minibatch:
            cnn.train(input_array, target_list)
            i = 0

    scorecard = [] 
    for i, correct_label in enumerate(test_label_list): 
        inputs = test_data_array[i, :, :, :]
        inputs = inputs.reshape(1, 28, 28, 1) 
        outputs = cnn.query(inputs) 
        #print(outputs)
        label = np.argmax(outputs) 
        #print(label, correct_label)

        if(label == int(correct_label)): 
            scorecard.append(1) 
        else: 
            scorecard.append(0)

    #print("test_T_F_list", scorecard) 
    scorecard_array = np.asarray(scorecard) 
    counter += 1
    print(counter)
    print("performance = ", scorecard_array.sum() / scorecard_array.size)
    
    
    
    mkdir(path + str(counter))
    cnn.saveData(path + str(counter) + '\\')






    #read_mat = np.loadtxt(filename, dtype=np.float64, delimiter=',')
    #print(read_mat) 



1
performance =  0.9553
2
performance =  0.969
3
performance =  0.9746
4
performance =  0.9553
5
performance =  0.9715
6
performance =  0.9753
7
performance =  0.9784
8
performance =  0.9828
9
performance =  0.9834
10
performance =  0.9878
11
performance =  0.9878
12
performance =  0.9877
13
performance =  0.9872
14
performance =  0.9885
15
performance =  0.9877
16
performance =  0.9876
17
performance =  0.9885
18
performance =  0.9887
19
performance =  0.9885
20
performance =  0.9895
21
performance =  0.9887
22
performance =  0.9882
23
performance =  0.9883
24
performance =  0.9885
25
performance =  0.9883
26
performance =  0.9873
27
performance =  0.9889
28
performance =  0.9902
29
performance =  0.9909
30
performance =  0.9898
31
performance =  0.9874
32
performance =  0.9877
33
performance =  0.9879
34
performance =  0.9895
35
performance =  0.9858
36
performance =  0.989
37
performance =  0.989
38
performance =  0.9899
39
performance =  0.9898


In [90]:
path = 'D:\\python\\neural_network\\parameter_zero\\count_'
counter = 29
cnn.loadData(path + str(counter) + '\\')